In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam

In [2]:
# model parameters
reuse = False
block = NNConv_block
head = cat3Head
data = '../Data/{}_data_ACSF_expand.pickle'
batch_size = 32
dim = 128
epochs = 20
clip = 2
layer1 = 2
layer2 = 2
factor = 2
lr = 1e-4

#### Train base

In [3]:
train_dl,val_dl = get_data(data,batch_size)

In [3]:
model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

In [11]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)

In [6]:
model,train_loss_perType,val_loss_perType = train(opt,model,epochs,train_dl,val_dl,paras,clip)

epoch:0, train_loss: +0.815, val_loss: +0.211, 
train_vector: +2.90|+1.43|+0.52|-0.01|+0.14|+0.83|+0.65|+0.06, 
val_vector  : +1.35|+0.72|+0.20|-0.67|-0.48|+0.69|+0.38|-0.50

epoch:1, train_loss: +0.061, val_loss: +0.032, 
train_vector: +0.98|+0.45|+0.07|-0.73|-0.60|+0.62|+0.33|-0.61, 
val_vector  : +1.03|+0.31|+0.05|-0.59|-0.79|+0.66|+0.28|-0.70

epoch:2, train_loss: -0.124, val_loss: -0.141, 
train_vector: +0.71|+0.18|-0.09|-0.96|-0.82|+0.55|+0.24|-0.79, 
val_vector  : +0.65|+0.28|-0.17|-0.58|-1.08|+0.54|+0.16|-0.92

epoch:3, train_loss: -0.230, val_loss: -0.261, 
train_vector: +0.59|+0.04|-0.19|-1.10|-0.95|+0.51|+0.17|-0.91, 
val_vector  : +0.50|+0.35|-0.27|-1.23|-1.05|+0.49|+0.11|-0.99

epoch:4, train_loss: -0.305, val_loss: -0.385, 
train_vector: +0.49|-0.07|-0.25|-1.18|-1.05|+0.48|+0.13|-0.99, 
val_vector  : +0.38|-0.27|-0.31|-1.16|-1.22|+0.48|+0.07|-1.05

epoch:5, train_loss: -0.358, val_loss: -0.424, 
train_vector: +0.44|-0.12|-0.30|-1.26|-1.12|+0.46|+0.09|-1.05, 
val_vector  :

In [8]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [13]:
save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

#### train on each type

In [3]:
model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

In [6]:
lr_type =1e-4/2
epochs_type = 10

In [4]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr_type)

In [9]:
file_name = !ls ../Model/
file_name = file_name[1]

In [11]:
for i in range(8):
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)

    checkpoint = torch.load('../Model/'+file_name)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt.load_state_dict(checkpoint['optimizer_state_dict'])

    for para in opt.param_groups:
        para['lr'] = lr_type
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)

    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i))

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i))


Training type: 0
epoch:0, train_loss: +0.069, val_loss: -0.064, 
train_vector: +0.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -0.026, val_loss: +0.310, 
train_vector: -0.03|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.31|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.072, val_loss: -0.131, 
train_vector: -0.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.105, val_loss: +0.163, 
train_vector: -0.11|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.132, val_loss: +0.020, 
train_vector: -0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.02|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:5, train_loss: -0.155, val_loss: +0.028, 
train_vector: -0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.

epoch:6, train_loss: -1.854, val_loss: -1.846, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.85|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.85|+0.00|+0.00|+0.00

epoch:7, train_loss: -1.874, val_loss: -1.881, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.87|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.88|+0.00|+0.00|+0.00

epoch:8, train_loss: -1.894, val_loss: -1.953, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.89|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.95|+0.00|+0.00|+0.00

epoch:9, train_loss: -1.909, val_loss: -1.859, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.91|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.86|+0.00|+0.00|+0.00

Training completed in 189.78208541870117s

Training type: 5
epoch:0, train_loss: +0.124, val_loss: +0.023, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.12|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.02|+0.00|+0.00

epoch:1, train_loss: -0.016, val_loss: -0.092, 
train_vector: +0.

#### Train on each type from start

In [3]:
head = cat3Head_type

In [4]:
lr_type =1e-4
epochs_type = 25

In [5]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # init model
    model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)

    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')


Training type: 0
epoch:0, train_loss: +0.896, val_loss: +0.131, 
train_vector: +0.90|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.113, val_loss: -0.043, 
train_vector: +0.11|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.04|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.023, val_loss: -0.134, 
train_vector: -0.02|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.102, val_loss: -0.200, 
train_vector: -0.10|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.20|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.166, val_loss: -0.253, 
train_vector: -0.17|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.25|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:5, train_loss: -0.216, val_loss: -0.243, 
train_vector: -0.22|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.

epoch:22, train_loss: -1.055, val_loss: -1.011, 
train_vector: +0.00|-1.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-1.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:23, train_loss: -1.069, val_loss: -1.071, 
train_vector: +0.00|-1.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-1.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:24, train_loss: -1.084, val_loss: -0.851, 
train_vector: +0.00|-1.08|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-0.85|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 497.0621814727783s

Training type: 2
epoch:0, train_loss: -0.254, val_loss: -0.609, 
train_vector: +0.00|+0.00|-0.25|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|-0.61|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -0.709, val_loss: -0.816, 
train_vector: +0.00|+0.00|-0.71|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|-0.82|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.877, val_loss: -0.972, 
train_vector: +

epoch:18, train_loss: -2.149, val_loss: -1.951, 
train_vector: +0.00|+0.00|+0.00|-2.15|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.95|+0.00|+0.00|+0.00|+0.00

epoch:19, train_loss: -2.153, val_loss: -2.233, 
train_vector: +0.00|+0.00|+0.00|-2.15|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-2.23|+0.00|+0.00|+0.00|+0.00

epoch:20, train_loss: -2.179, val_loss: -2.084, 
train_vector: +0.00|+0.00|+0.00|-2.18|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-2.08|+0.00|+0.00|+0.00|+0.00

epoch:21, train_loss: -2.203, val_loss: -2.141, 
train_vector: +0.00|+0.00|+0.00|-2.20|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-2.14|+0.00|+0.00|+0.00|+0.00

epoch:22, train_loss: -2.210, val_loss: -2.266, 
train_vector: +0.00|+0.00|+0.00|-2.21|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-2.27|+0.00|+0.00|+0.00|+0.00

epoch:23, train_loss: -2.217, val_loss: -2.266, 
train_vector: +0.00|+0.00|+0.00|-2.22|+0.00|+0.00|+0.00|+0.00, 
val_vec

epoch:14, train_loss: -1.138, val_loss: -1.105, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.14|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.10|+0.00|+0.00

epoch:15, train_loss: -1.165, val_loss: -1.115, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.16|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.11|+0.00|+0.00

epoch:16, train_loss: -1.188, val_loss: -1.135, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.19|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.14|+0.00|+0.00

epoch:17, train_loss: -1.211, val_loss: -1.131, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.21|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.13|+0.00|+0.00

epoch:18, train_loss: -1.230, val_loss: -1.156, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.23|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.16|+0.00|+0.00

epoch:19, train_loss: -1.251, val_loss: -1.158, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.25|+0.00|+0.00, 
val_vec

epoch:10, train_loss: -2.197, val_loss: -2.032, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.20, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.03

epoch:11, train_loss: -2.235, val_loss: -2.106, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.24, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.11

epoch:12, train_loss: -2.271, val_loss: -2.117, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.27, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.12

epoch:13, train_loss: -2.306, val_loss: -2.156, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.31, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.16

epoch:14, train_loss: -2.339, val_loss: -2.173, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.34, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.17

epoch:15, train_loss: -2.369, val_loss: -2.200, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.37, 
val_vec

#### two stage bigger head

In [3]:
lr_type =1e-4/2
epochs_type = 10
file_name = "False_<class 'functions_refactor.schnet_block'>_<class 'functions_refactor.cat3Head'>__data_ACSF_expand.pickle_32_128_2_2_2_2_20_base.tar"

In [4]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # load base model
    model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    checkpoint = torch.load('../Model/'+file_name)
    model.load_state_dict(checkpoint['model_state_dict'])

    set_requires_grad(model, False)
    model.head = cat3Head_type(dim,8,data_dict[data]['edge_in4']).to('cuda:0')
    
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type) 
    # train head
    print('\nTraining type: {}'.format(i))
    print('\nTraining head')
    model,train_loss_perType,val_loss_perType = train(opt,model,5,train_dl,val_dl,paras,clip,True)
    
    # train all
    set_requires_grad(model, True)
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)    
    print('\nTraining all')
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,train_loss_perType,val_loss_perType)
    
    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 cat3Head_type,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')

    save_model(model,opt,reuse,block,cat3Head_type,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')


Training type: 0

Training head
epoch:0, train_loss: +0.589, val_loss: +0.061, 
train_vector: +0.59|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.008, val_loss: +0.214, 
train_vector: +0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.21|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.005, val_loss: +0.116, 
train_vector: -0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.12|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.008, val_loss: +0.121, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.12|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.010, val_loss: +0.239, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.24|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 85.70982599258423s

Training all
epoch:5, train_loss: +0.050, val_loss


Training type: 3

Training head
epoch:0, train_loss: -1.242, val_loss: -1.765, 
train_vector: +0.00|+0.00|+0.00|-1.24|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.76|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -1.703, val_loss: -1.774, 
train_vector: +0.00|+0.00|+0.00|-1.70|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.77|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -1.733, val_loss: -1.693, 
train_vector: +0.00|+0.00|+0.00|-1.73|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.69|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -1.748, val_loss: -1.865, 
train_vector: +0.00|+0.00|+0.00|-1.75|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.86|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -1.766, val_loss: -1.651, 
train_vector: +0.00|+0.00|+0.00|-1.77|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.65|+0.00|+0.00|+0.00|+0.00

Training completed in 87.28697752952576s

Training all
epoch:5, train_loss: -1.664, val_loss


Training type: 6

Training head
epoch:0, train_loss: -0.193, val_loss: -0.417, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.19|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.42|+0.00

epoch:1, train_loss: -0.423, val_loss: -0.460, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.42|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.46|+0.00

epoch:2, train_loss: -0.449, val_loss: -0.413, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.45|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.41|+0.00

epoch:3, train_loss: -0.474, val_loss: -0.513, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.47|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.51|+0.00

epoch:4, train_loss: -0.493, val_loss: -0.516, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.49|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.52|+0.00

Training completed in 84.60186243057251s

Training all
epoch:5, train_loss: -0.620, val_loss

### make submissions

In [14]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

From start

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_start')

2Stage

In [3]:
head = cat3Head_type

In [4]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_2stage')